In [ ]:
import openai
import os

# Configurar cliente de Azure OpenAI
client = openai.AzureOpenAI(
    api_key="9c04b423bfa64c82a178f71365d15094",
    api_version="2024-05-01-preview",
    azure_endpoint="https://cursillogenai.openai.azure.com/"
)

# Crear funciones/tools
tools = [
    {
        "type": "function",
        "function": {
            "name": "CheckStatus",
            "description": "Muestra al usuario el estado actual de sus solicitudes.",
            "parameters": {
                "type": "object",
                "properties": {},
                "required": []
            }
        }
    },
    {
        "type": "function",
        "function": {
  "name": "CreateSolicitudWifi",
  "description": "Crea una solicitud de red WiFi",
  "parameters": {
    "type": "object",
    "properties": {
      "tipo_red": {
        "type": "string",
        "enum": ["corporativa", "invitados"],
        "description": "Tipo de red WiFi a solicitar"
      },
      "destinatarios": {
        "type": "string",
        "description": "Correos de los destinatarios de la peticion separados por comas. Ej: 'juan@megamedia.es, ana@mediaset.es' (si el destinatario es el propio usuario inluyelo como 'user')"
      },
      "observaciones": {
        "type": "string",
        "description": "Observaciones adicionales",
        "default": ""
      }
    },
    "required": ["tipo_red"]
  }
}
    },
    {
        "type": "function",
        "function": {
            "name": "CreateSolicitudOfimatica",
            "description": "Solicita material o accesorios de ofimática.",
            "parameters": {
                "type": "object",
                "properties": {
                    "tipo_accesorio": {
                        "type": "string",
                         "enum": [
                            "Monitor",
                            "Ratón",
                            "Teclado",
                            "Altavoces",
                            "Cascos",
                            "Pendrive",
                            "XDCAM",
                            "Escaner",
                            "Impresora B/N (láser)",
                            "Ipad/Tablet",
                            "otro material (indicar descripción)",
                            "Camara Web",
                            "Mochila para portátil"
                            ],
                        "description": "Tipo de accesorio solicitado"
                    },
                    "cc": {
                        "type": "string",
                        "description": "Centro de coste"
                    },
                    "destinatarios": {
                        "type": "string",
                        "description": "Correos de los destinatarios de la peticion separados por comas. Ej: 'juan@megamedia.es, ana@mediaset.es' (si el destinatario es el propio usuario inluyelo como 'user')"
                    },
                    "descripcion": {
                        "type": "string",
                        "description": "Descripción del accesorio"
                    },
                    "fecha_necesidad": {
                        "type": "string",
                        "description": "Fecha en la que se necesita el accesorio (formato: YYYY-MM-DD)"
                    },
                    "cantidad": {
                        "type": "integer",
                        "description": "Cantidad del accesorio solicitado"
                    }
                },
                "required": ["tipo_accesorio", "cc", "descripcion", "fecha_necesidad", "cantidad","destinatarios"]
            }
        }
    }
]

# Crear el assistant
assistant = client.beta.assistants.create(
    name="SolicitudesOnlineDemo",
    instructions=(
        "Eres un asistente para gestionar solicitudes internas. "
        "Puedes crear solicitudes de acceso a WiFi, pedidos de accesorios de ofimática, "
        "y consultar el estado actual de las solicitudes del usuario."
    ),
    tools=tools,
    model="gpt-4o-mini"  # o el nombre del deployment si es diferente
)

print("✅ Assistant creado con ID:", assistant.id)


✅ Assistant creado con ID: asst_oUq6FC9tPguNpItCe7W3IJ2B


In [13]:
import openai
import os

# Configurar cliente de Azure OpenAI
client = openai.AzureOpenAI(
    api_key="9c04b423bfa64c82a178f71365d15094",
    api_version="2024-05-01-preview",
    azure_endpoint="https://cursillogenai.openai.azure.com/"
)

# Crear funciones/tools
tools = [
    {"type": "file_search"},
    {
        "type": "function",
        "function": {
            "name": "CheckStatus",
            "description": "Muestra al usuario el estado actual de sus solicitudes.",
            "parameters": {
                "type": "object",
                "properties": {},
                "required": []
            }
        }
    },
    {
  "type": "function",
  "function": {
    "name": "CreateSolicitudWifi",
    "description": "Crea una solicitud de red WiFi",
    "parameters": {
      "type": "object",
      "properties": {
        "tipo_red": {
          "type": "string",
          "enum": ["corporativa", "invitados"],
          "description": "Tipo de red WiFi a solicitar"
        },
        "destinatarios_internos": {
          "type": "array",
          "description": "Correos de los destinatarios internos. Ej: ['juan@megamedia.es', 'ana@mediaset.es']. Incluir 'user' si el propio usuario es uno de ellos",
          "items": {
            "type": "string",
            "format": "email"
          }
        },
        "destinatarios_externos": {
          "type": "array",
          "description": "Lista de personas externas sin correo corporativo, con sus datos",
          "items": {
            "type": "object",
            "properties": {
              "nombre": { "type": "string","description": "Nombre del destinatario externo."},
              "apellidos": { "type": "string","description": "Apellidos del destinatario externo" },
              "departamento": { "type": "string", "description": "Departamento donde trabaja el destinatario externo" },
              "empresa": { "type": "string", "description": "Empresa donde trabaja el destinatario externo" },
              "centro": { "type": "string", "description": "Centro asignado al destinatario externo","enum": ["Alicante", "Bilbao","Barcelona","Fuencarral","Sevilla","Villaviciosa"]},
              "edificio": { "type": "string", "description": "Edificio asignado al destinatario externo (los centros de Alicante, Bilbao, Barcelona y Sevilla no tienen edificos para estos esta la opcion 'ninguno')","enum":["A", "B","C","D","E","F","G","H","recepcion","caracolas","ninguno"],"default": "ninguno" },
              "planta": { "type": "string", "description": "Planta asignada al destinatario externo (-1 se refiere a sotano, 0 a la planta baja, los centros de Alicante, Bilbao, Barcelona y Sevilla no tienen edificos y por tanto no tienen plantas para estos esta la opcion 'ninguno')","enum":["-1","0","1","2","3","4","5","ninguno"], "default": "ninguno"}
            },
            "required": ["nombre", "empresa","apellidos","departamento","centro","edificio","planta"]
          }
        },
        "observaciones": {
          "type": "string",
          "description": "Observaciones adicionales",
          "default": ""
        }
      },
      "required": ["tipo_red"]
    }
  }
},
 {
  "type": "function",
  "function": {
    "name": "CreateSolicitudVPN",
    "description": "Crea una solicitud para solicitar acceso a la VPN",
    "parameters": {
      "type": "object",
      "properties": {
        "destinatarios_internos": {
          "type": "array",
          "description": "Correos de los destinatarios internos. Ej: ['juan@megamedia.es', 'ana@mediaset.es']. Incluir 'user' si el propio usuario es uno de ellos",
          "items": {
            "type": "string",
            "format": "email"
          }
        },
        "destinatarios_externos": {
          "type": "array",
          "description": "Lista de personas externas sin correo corporativo, con sus datos",
          "items": {
            "type": "object",
            "properties": {
              "nombre": { "type": "string","description": "Nombre del destinatario externo."},
              "apellidos": { "type": "string","description": "Apellidos del destinatario externo" },
              "departamento": { "type": "string", "description": "Departamento donde trabaja el destinatario externo" },
              "empresa": { "type": "string", "description": "Empresa donde trabaja el destinatario externo" },
              "centro": { "type": "string", "description": "Centro asignado al destinatario externo","enum": ["Alicante", "Bilbao","Barcelona","Fuencarral","Sevilla","Villaviciosa"]},
              "edificio": { "type": "string", "description": "Edificio asignado al destinatario externo (los centros de Alicante, Bilbao, Barcelona y Sevilla no tienen edificos para estos esta la opcion 'ninguno')","enum":["A", "B","C","D","E","F","G","H","recepcion","caracolas","ninguno"],"default": "ninguno" },
              "planta": { "type": "string", "description": "Planta asignada al destinatario externo (-1 se refiere a sotano, 0 a la planta baja, los centros de Alicante, Bilbao, Barcelona y Sevilla no tienen edificos y por tanto no tienen plantas para estos esta la opcion 'ninguno')","enum":["-1","0","1","2","3","4","5","ninguno"], "default": "ninguno"}
            },
            "required": ["nombre", "empresa","apellidos","departamento","centro","edificio","planta"]
          }
        },
        "observaciones": {
          "type": "string",
          "description": "Observaciones adicionales",
          "default": ""
        }
      },
    }
  }
},
 {
  "type": "function",
  "function": {
    "name": "CreateSolicitudOffice365",
    "description": "Crea una solicitud para solicitar office 365",
    "parameters": {
      "type": "object",
      "properties": {
        "destinatarios_internos": {
          "type": "array",
          "description": "Correos de los destinatarios internos. Ej: ['juan@megamedia.es', 'ana@mediaset.es']. Incluir 'user' si el propio usuario es uno de ellos",
          "items": {
            "type": "string",
            "format": "email"
          }
        },
        "destinatarios_externos": {
          "type": "array",
          "description": "Lista de personas externas sin correo corporativo, con sus datos",
          "items": {
            "type": "object",
            "properties": {
              "nombre": { "type": "string","description": "Nombre del destinatario externo."},
              "apellidos": { "type": "string","description": "Apellidos del destinatario externo" },
              "departamento": { "type": "string", "description": "Departamento donde trabaja el destinatario externo" },
              "empresa": { "type": "string", "description": "Empresa donde trabaja el destinatario externo" },
              "centro": { "type": "string", "description": "Centro asignado al destinatario externo","enum": ["Alicante", "Bilbao","Barcelona","Fuencarral","Sevilla","Villaviciosa"]},
              "edificio": { "type": "string", "description": "Edificio asignado al destinatario externo (los centros de Alicante, Bilbao, Barcelona y Sevilla no tienen edificos para estos esta la opcion 'ninguno')","enum":["A", "B","C","D","E","F","G","H","recepcion","caracolas","ninguno"],"default": "ninguno" },
              "planta": { "type": "string", "description": "Planta asignada al destinatario externo (-1 se refiere a sotano, 0 a la planta baja, los centros de Alicante, Bilbao, Barcelona y Sevilla no tienen edificos y por tanto no tienen plantas para estos esta la opcion 'ninguno')","enum":["-1","0","1","2","3","4","5","ninguno"], "default": "ninguno"}
            },
            "required": ["nombre", "empresa","apellidos","departamento","centro","edificio","planta"]
          }
        },
        "observaciones": {
          "type": "string",
          "description": "Observaciones adicionales",
          "default": ""
        }
      },
    }
  }
},
    {
        "type": "function",
        "function": {
            "name": "CreateSolicitudOfimatica",
            "description": "Solicita material o accesorios de ofimática.",
            "parameters": {
                "type": "object",
                "properties": {
                    "tipo_accesorio": {
                        "type": "string",
                         "enum": [
                            "Monitor",
                            "Ratón",
                            "Teclado",
                            "Altavoces",
                            "Cascos",
                            "Pendrive",
                            "XDCAM",
                            "Escaner",
                            "Impresora B/N (láser)",
                            "Ipad/Tablet",
                            "otro material (indicar descripción)",
                            "Camara Web",
                            "Mochila para portátil"
                            ],
                        "description": "Tipo de accesorio solicitado"
                    },
                    "cc": {
                        "type": "string",
                        "description": "Centro de coste"
                    },
                    "destinatarios_internos": {
                    "type": "array",
                    "description": "Correos de los destinatarios internos. Ej: ['juan@megamedia.es', 'ana@mediaset.es']. Incluir 'user' si el propio usuario es uno de ellos",
                    "items": {
                        "type": "string",
                        "format": "email"
                    }
                    },
                    "destinatarios_externos": {
                    "type": "array",
                    "description": "Lista de personas externas sin correo corporativo, con sus datos",
                    "items": {
                        "type": "object",
                        "properties": {
                        "nombre": { "type": "string","description": "Nombre del destinatario externo."},
                        "apellidos": { "type": "string","description": "Apellidos del destinatario externo" },
                        "departamento": { "type": "string", "description": "Departamento donde trabaja el destinatario externo" },
                        "empresa": { "type": "string", "description": "Empresa donde trabaja el destinatario externo" },
                        "centro": { "type": "string", "description": "Centro asignado al destinatario externo","enum": ["Alicante", "Bilbao","Barcelona","Fuencarral","Sevilla","Villaviciosa"]},
                        "edificio": { "type": "string", "description": "Edificio asignado al destinatario externo (los centros de Alicante, Bilbao, Barcelona y Sevilla no tienen edificos para estos esta la opcion 'ninguno')","enum":["A", "B","C","D","E","F","G","H","recepcion","caracolas","ninguno"],"default": "ninguno" },
                        "planta": { "type": "string", "description": "Planta asignada al destinatario externo (-1 se refiere a sotano, 0 a la planta baja, los centros de Alicante, Bilbao, Barcelona y Sevilla no tienen edificos y por tanto no tienen plantas para estos esta la opcion 'ninguno')","enum":["-1","0","1","2","3","4","5","ninguno"], "default": "ninguno"}
                        },
                        "required": ["nombre", "empresa","apellidos","departamento","centro","edificio","planta"]
                    }
                    },
                    "descripcion": {
                        "type": "string",
                        "description": "Descripción del accesorio"
                    },
                    "fecha_necesidad": {
                        "type": "string",
                        "description": "Fecha en la que se necesita el accesorio (formato: YYYY-MM-DD). Hoy es Martes, 2025-06-25"
                    },
                    "cantidad": {
                        "type": "integer",
                        "description": "Cantidad del accesorio solicitado"
                    }
                },
                "required": ["tipo_accesorio", "cc", "descripcion", "fecha_necesidad", "cantidad","destinatarios"]
            }
        }
    }
]

# Crear el assistant
assistant = client.beta.assistants.create(
    name="SolicitudesOnlineDemo",
    instructions=(
        "Eres un asistente para gestionar solicitudes internas. Tienes informacion sobre los sistemas de la empresa como la VPN y otras preguntas tipicas. Debes ayudar a los usuarios resolviendo esas dudas."
        "Puedes crear solicitudes de acceso a WiFi, pedidos de accesorios de ofimática, "
        "y consultar el estado actual de las solicitudes del usuario. ALWAYS REPLY IN THE SAME LANGUAGE!!!"
    ),
    tools=tools,
    tool_resources={
        "file_search": {
            "vector_store_ids": ["vs_7UY32DrRdUekNAMDFnhuG2Ng"]  # Reemplaza por el ID de tu vector store
        }
    },
    model="gpt-4o"  # o el nombre del deployment si es diferente
)

print("✅ Assistant creado con ID:", assistant.id)


✅ Assistant creado con ID: asst_zQbMRNefcgFT6DWKnSegZgLV
